# Optimizacija fazi kontrolera za vožnju automobila korišćenjem genetskog algoritma

## Uvod

Cilj ovog rada je bila optimizacija fazi kontrolera za vožnju korišćenjem genetskog algoritma. Kontroler je upravljao automobilom koji se kretao po otvorenom ili zatvorenom putu, u zavisnosti od zadatih opcija.  

U radu je prvo pokazano da je moguće konstruisati fazi kontroler koji će pravilno upravljati automobilom, a zatim je pokazano i da je moguće optimizovati nasumično izgenerisan fazi sistem uz pomoć genetskog algoritma tako da pravilno upravlja automobilom.  

Automobilom se upravlja pravilno ako se automobil ne sudara sa ivicama puta i ako se kreće po sredini puta, koliko god je to moguće.

Automobil je testiran na dve vrste puta. Prvi tip puta je krivudav put koji je izgenerisan sinusiodom, a drugi tip puta je bio zatvoren kružni put izgenerisan pomoću konveksnog omotača nasumično odabranog skupa koordinata.

## Fazi sistem

Fazi sistem se sastoji iz 3 ulaza i 2 izlaza. Ulazi su predstavljali udaljenost od kraja puta, a izlazi su predstavljali promenu ugla i brzine.

### Ulaz

Postoje 3 ulazne funkcije:
- `left_sensor`
- `right_sensor`
- `front_sensor`

Svaka od njih ima 4 iste funkcije pripadnosti:  
- `close`
- `midrange`
- `far`
- `very far`

One označavaju koliko smo udaljenosti od svake od ivica puta. 

### Izlaz

Postoje 2 izlazne funkcije:
- `velocity`
- `angle`

`velocity` označava promenu brzine, dok `angle` označava promenu ugla skretanja volana.

`velocity` ima 4 funkcije pripadnosti:
- `low`
- `middle`
- `high`
- `very high`

`angle` ima 5 funkcija pripadnosti:
- `hard right`
- `right`
- `forward`
- `left`
- `hard left`

Sve funkcije pripadnosti su trapeznog oblika, sem funkcija na kraju i na početku intervala, čiji je oblik najsličniji ramp funkciji. Takođe, sve funkcije su imaju interval iz kog smeju da uzimaju vrednosti. Funkcije se mogu lepo videti u sledećem rečniku, koji se koristi pri generisanju nasumičnog fazi sistema:


In [ ]:
ALL_FUZZY_FUNCS = {
    "left_sensor": {
        "name": "left_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "right_sensor": {
        "name": "right_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "front_sensor": {
        "name": "front_sensor",
        "left_boundary": 0,
        "right_boundary": 20,
        "mf_names": ["close", "midrange", "far", "very far"],
        "is_input": True,
    },
    "velocity": {
        "name": "velocity",
        "left_boundary": 1,
        "right_boundary": 20,
        "mf_names": ["low", "middle", "high", "very high"],
        "is_input": False,
    },
    "angle": {
        "name": "angle",
        "left_boundary": -45,
        "right_boundary": 45,
        "mf_names": ["hard right", "right", "forward", "left", "hard left"],
        "is_input": False,
    }
}